In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, beta, uniform
import sys


x_values = np.linspace(-8, 10, 500)  # For plotting the curve

m0 = 0
s0 = 2.5
pdf0_values = norm.pdf(x_values, loc=m0, scale=s0)


def pdf0(x):
    return norm.pdf(x, loc=m0, scale=s0)


m1 = 5
s1 = 1
pdf1_values = norm.pdf(x_values, loc=m1, scale=s1)


def pdf1(x):
    return norm.pdf(x, loc=m1, scale=s1)


lik_ratios = pdf1_values / pdf0_values
# beta_quantiles = np.linspace(0, 1, 5)
beta_quantiles = [0, 1 / 3, 0.5, 0.7, 0.8, 0.9, 1]
betas = [np.quantile(lik_ratios, q) for q in beta_quantiles]
betas[-1] = np.inf

betas = betas
beta_quantiles = beta_quantiles

n_target = 10000


# print(f'betas : {betas}')

for i, beta in enumerate(betas):
    pdf1_constrained_values = np.where(
        lik_ratios < beta, pdf1_values, beta * pdf0_values
    )
    x_constraint_solution_ = np.where(lik_ratios < beta, True, False)

    if np.sum(x_constraint_solution_) == len(x_constraint_solution_):
        ## If True everywhere:
        idx_x_left = -1
        idx_x_right = -1
    else:
        idx_x_left = np.argmin(x_constraint_solution_)

        idx_x_right = (
            x_constraint_solution_.size - np.argmin(x_constraint_solution_[::-1]) - 1
        )
        print(f"l, r : {idx_x_left}, {idx_x_right}")

    x_constraint_solution = x_values[np.sum(x_constraint_solution_) - 1]
    if i == len(betas) - 1:
        x_constraint_solution = -4

    # if (beta_quantiles[i] <= 0.7):
    # norm_const_est
    # weight_else = 0
    norm_const_est_vals = []
    if beta <= 0.7:
        ## Rejection sampling from safe dist
        # n_proposal = 10000
        accepted_samples = []
        # for j in range(n_proposal):
        while len(accepted_samples) < n_target:
            u = uniform.rvs()
            safe_sample = norm.rvs(loc=m0, scale=s0)
            # if (u < pdf1(safe_samples[j])/(beta * pdf0(safe_samples[j]))):
            # print(pdf1(safe_sample))
            scaled_ratio = pdf1(safe_sample) / (beta * pdf0(safe_sample))
            if scaled_ratio < 1:
                norm_const_est_vals.append(pdf1(safe_sample) / pdf0(safe_sample))
                # norm_const_est_vals.append(pdf1(safe_sample))
            else:
                norm_const_est_vals.append(beta)
            if u < scaled_ratio:
                accepted_samples.append(safe_sample)
            # else:
            #     print("rejected")
            # print('accepted')
        # print(f"norm_const_est_vals : {weight_if}")
        # print(f"norm_const_est_vals : {weight_else}")
        norm_const_est = np.mean(norm_const_est_vals)

    else:
        # Rejection sampling from unconst dist
        # n_proposal = 10000
        # unconstrained_samples = norm.rvs(loc=m1, scale=s1, size=n_proposal)
        accepted_samples = []
        # for j in range(n_proposal):
        while len(accepted_samples) < n_target:
            u = uniform.rvs()
            unconstrained_sample = norm.rvs(loc=m1, scale=s1)
            # if (u < pdf0(unconstrained_samples[j]) * beta / pdf1(unconstrained_samples[j])):
            scaled_ratio = (
                pdf0(unconstrained_sample) * beta / pdf1(unconstrained_sample)
            )
            # print(scaled_ratio)
            if scaled_ratio >= 1:
                norm_const_est_vals.append(1)
            else:
                norm_const_est_vals.append(
                    np.clip(
                        pdf0(unconstrained_sample) * beta / pdf1(unconstrained_sample),
                        a_min=None,
                        a_max=99999999,
                    )
                )
                # norm_const_est_vals.append(beta)

            if u < scaled_ratio:
                accepted_samples.append(unconstrained_sample)
                # print('accepted')
            # else:
            #     print("rejected")

        norm_const_est = np.mean(norm_const_est_vals)
    # if (len(norm_const_est_vals) > 0):
    #     # print(norm_const_est_vals)
    #     norm_const_est = np.mean(norm_const_est_vals)
    # else:
    #     norm_const_est = 1

    # print(f'pdf1_constrained_values : {pdf1_constrained_values}')
    # pdf1_constrained_values_normalized = pdf1_constrained_values
    norm_const = (
        np.sum(pdf1_constrained_values)
        * (np.max(x_values) - np.min(x_values))
        / len(x_values)
    )
    print(f"norm_const : {norm_const}")
    # pdf1_constrained_values_normalized = pdf1_constrained_values / norm_const
    pdf1_constrained_values_normalized = pdf1_constrained_values / norm_const_est
    print(f"norm_const_est : {norm_const_est}")
    # print(pdf1_constrained_values)

    print(f"beta : {beta}")

    if beta < max(betas):
        plt.axvspan(
            xmin=x_values[idx_x_left], xmax=x_values[idx_x_right], alpha=0.1, color="C2"
        )

    plt.hist(np.array(accepted_samples), bins=100, density=True, alpha=0.5)

    plt.plot(
        x_values,
        pdf0_values,
        label="Safe Policy",
        color="C2",
        linestyle=":",
        alpha=0.75,
        linewidth=4,
    )
    plt.plot(
        x_values,
        pdf1_values,
        label="Unconstrained Policy",
        color="C3",
        linestyle=":",
        alpha=0.75,
        linewidth=4,
    )
    plt.plot(
        x_values,
        pdf1_constrained_values_normalized,
        label="Constrained Policy",
        color="C0",
        alpha=0.75,
        linewidth=4,
    )

    if beta == np.inf:
        plt.title(
            rf"Unconstrained Policy (Q($\beta$) = {beta_quantiles[i]})", fontsize=16
        )
    elif beta == min(betas):
        plt.title(
            rf"Fully Constrained Policy (Q($\beta$) = {beta_quantiles[i]})", fontsize=16
        )
    else:
        plt.title(
            rf"Partially Constrained Policy (Q($\beta$) = {beta_quantiles[i]})",
            fontsize=16,
        )

    print(x_constraint_solution)
    # plt.axvline(x=x_constraint_solution, color='black', linestyle=':', label=r'x s.t. p_t(x) / p_{safe} == $\lambda$')
    if i == len(betas) - 1:
        plt.legend(loc=[0, 1.1], ncols=4, fontsize=12)
    plt.xlabel("X", fontsize=16)
    plt.ylabel("Density", fontsize=16)
    plt.savefig(f"./constrained_ex_beta{beta}.png")
    plt.show()

In [ ]:
samples = norm.rvs(loc=m0, scale=s0, size=10)
samples

In [ ]:
a = [1, 2, 3]
[*a]

In [ ]:
np.linspace(0, 1, 5)

In [ ]:
print(sys.float_info.min)

In [ ]:
data = pd.read_json(
    "/data/bucket/prinstea/llome/parent_output_20250901_temp1/smaller_pythia/smaller_pythia_sft_init_r0/gens_init_likelihood_100sample_2iter.jsonl",
    lines=True,
)
data

In [ ]:
## Pre-training initialization:
import numpy as np
import pandas as pd

for i in range(0, 5):
    # data = pd.read_json(f'/data/bucket/prinstea/llome/parent_output_20250901_temp1/smaller_pythia/smaller_pythia_sft_init_r{i}/gens_init_likelihood_100sample_2iter.jsonl', lines = True)
    data = pd.read_json(
        f"/data/bucket/prinstea/llome/parent_output_20250901_temp1/smaller_pythia/smaller_pythia_sft_init_r{i}/gens_init_likelihood_100sample_2iter_temp1.0_10seqs.jsonl",
        lines=True,
    )
    # num_inf = len(data[data['score']==np.inf])
    num_infeasible = len(data[data["score"].isna()]) + len(
        data[data["score"] == np.inf]
    )
    num_in_training = len(data[data["in_dataset"]])

    n = len(data)
    print(f"r{i}, n : {n}, frac infeasible  : {num_infeasible / n}")
    print(f"r{i}, n : {n}, frac in training : {num_in_training / n} \n")

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {"col1": range(1), "col2": [f"val_{i}" for i in range(1)]}
df = pd.DataFrame(data)

# Set a random_state for reproducibility
# This ensures that the same random split is generated every time the code is run
random_seed = 42

# Define the fraction for the first part (e.g., 80% for training)
train_frac = 0.500001

# Create the training set by sampling the specified fraction
train_df = df.sample(frac=train_frac, random_state=random_seed)

# Create the testing set by dropping the rows that are in the training set
test_df = df.drop(train_df.index)

print("Training DataFrame shape:", train_df.shape)
print("Testing DataFrame shape:", test_df.shape)

In [ ]:
# data = pd.read_json('/data/bucket/prinstea/llome/parent_output/smaller_pythia/smaller_pythia_sft_r6/gens_likelihood_1sample_10iter_temp1.0_10seqs.jsonl', lines=True)
import pandas as pd
import numpy as np

## Extrapolation:
for i in range(25):
    # data = pd.read_json(f'/data/bucket/prinstea/llome/parent_output_20250901_temp1/smaller_pythia/smaller_pythia_sft_r{i}_20250901/gens_likelihood_10sample_10iter.jsonl', lines = True)
    # data = pd.read_json(f'/data/bucket/prinstea/llome/parent_output_20250904_moreSFT_100s/smaller_pythia/smaller_pythia_sft_r{i}/gens_likelihood_100sample_1iter.jsonl', lines=True)
    # data = pd.read_json(f'/data/bucket/prinstea/llome/parent_output_20250904_moreSFT_100s/smaller_pythia/smaller_pythia_sft_r{i}/gens_likelihood_25sample_1iter.jsonl', lines=True)
    data = pd.read_json(
        f"/data/bucket/prinstea/llome/parent_output_20250904_moreSFT_10s/smaller_pythia/smaller_pythia_sft_r{i}/gens_likelihood_10sample_1iter.jsonl",
        lines=True,
    )

    # num_inf = len(data[data['score']==np.inf])
    num_infeasible = len(data[data["score"].isna()]) + len(
        data[data["score"] == np.inf]
    )
    num_in_training = len(data[data["in_dataset"]])
    # display(data.iloc[0:2,:])

    n = len(data)
    print(f"r{i}, n : {n}, frac infeasible  : {num_infeasible / n}")
    print(f"r{i}, n : {n}, frac in training : {num_in_training / n} \n")

# data

In [ ]:
# data = pd.read_json('/data/bucket/prinstea/llome/parent_output/smaller_pythia/smaller_pythia_gpt/gens_init_likelihood_1000sample_10iter.jsonl', lines=True)
# data = pd.read_json('/data/bucket/prinstea/llome/parent_output/smaller_pythia/smaller_pythia_gpt/gens_init_likelihood_1000sample_10iter_temp1.0_10seqs.jsonl', lines=True)
# data = pd.read_json('/data/bucket/prinstea/llome/parent_output/smaller_pythia/gpt_unif_dense_neighborhood_pairs_xthres0.25_maxinfs0.1_30nn.jsonl', lines=True)
# data
data = pd.read_json(
    "/data/bucket/prinstea/llome/parent_output/smaller_pythia/smaller_pythia_sft_r1/gens_likelihood_100sample_10iter_temp1.0_10seqs.jsonl",
    lines=True,
)
# num_inf = len(data[data['score'].isna()]) + len(data[data['score']==np.inf])

n = len(data)
print(n)
# print(f'r{i}, frac infeasible : {num_inf/n}')
# data["in_dataset"].sum()
print(f"Frac particles in training data : {len(data[data['in_dataset']]) / n}")
data

In [ ]:
import numpy as np

a = np.array(
    [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.9999757,
        0.00268334,
        0.9996437,
        0.9984262,
        0.9999994,
        0.96831906,
        0.9999993,
        0.9276532,
        0.9999999,
        0.9457009,
        1.0,
        0.9989404,
        1.0,
        0.7990209,
        1.0,
        0.9945381,
        0.99999654,
        0.9836394,
        1.0,
        0.99783605,
        1.0,
        0.939762,
        0.99999964,
        0.999624,
        1.0,
        0.9961732,
        1.0,
        0.9590705,
        1.0,
        0.9468553,
        0.99999976,
        0.99964845,
        0.99999964,
        0.9756631,
        0.9999999,
        0.9997564,
        1.0,
        0.9998136,
        0.99999976,
        0.9795368,
        0.99999845,
        0.8060646,
        0.99999905,
        0.99828523,
        0.999997,
        0.9507775,
        0.99999905,
        0.9964759,
        1.0,
        0.98720807,
        0.99999976,
        0.9977544,
        0.9998641,
        0.9933022,
        1.0,
        0.49467334,
        0.99999595,
        0.8103291,
        0.9999355,
        0.9646651,
        0.9999999,
        0.99300814,
        0.9999968,
        0.82099724,
        0.9999244,
    ]
)

b = np.array(
    [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.99997985,
        0.00346163,
        0.9996593,
        0.99715245,
        0.9999993,
        0.97442675,
        0.99999917,
        0.9453344,
        0.9999999,
        0.9257733,
        1.0,
        0.9991731,
        1.0,
        0.8534383,
        1.0,
        0.98922884,
        0.99999654,
        0.9874391,
        1.0,
        0.9980338,
        1.0,
        0.94244903,
        0.9999999,
        0.9993235,
        1.0,
        0.99135756,
        1.0,
        0.95656496,
        1.0,
        0.9656669,
        0.9999999,
        0.99915934,
        0.9999994,
        0.9750972,
        0.9999999,
        0.9995208,
        1.0,
        0.99948394,
        0.9999987,
        0.965408,
        0.9999962,
        0.8454996,
        0.99999905,
        0.9985127,
        0.99999774,
        0.9480506,
        0.99999833,
        0.98997724,
        1.0,
        0.9446316,
        0.9999999,
        0.9941362,
        0.9999126,
        0.9825868,
        1.0,
        0.60311216,
        0.9999974,
        0.90260494,
        0.9999864,
        0.9594176,
        1.0,
        0.9928461,
        0.9999989,
        0.78914386,
        0.99979085,
    ]
)

np.where((a == 0) & (b > 0))

In [ ]:
import torch

generated_suffix = torch.empty((1, 0))
generated_suffix.shape

In [ ]:
from typing import List


def func(a: List[str]):
    return a


a = ["a", "b"]
func(a)

In [ ]:
data_fp_list_str = "\\["
data_fp_list_str += "asdf"
data_fp_list_str